# Carga de Datos


## .xls / HTML a .csv
Los datos del Mineduc se exportan como archivos .xls, realmente teniendo contenido de HTML. Debido a eso, debemos de parsear el archivo HTML e identificar la tabla correcta que contiene los datos. Luego, podemos exportar los datos a su archivo .csv correspondiente, utilizando el valor de la columna 'Departamento' para nombrarlo. Adicionalmente, los archivos utilizan un encoding diferente al estándar utf-8, por lo cual vamos a especificarlo al momento de leer los archivos HTML.

In [49]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
import re


def parse_html_excel_file(file_path):
    file_path = Path(file_path)

    try:
        tables = pd.read_html(str(file_path), encoding="iso-8859-1")

        if not tables:
            return {"success": False, "error": "No tables found in HTML"}

        required_headers = ["CODIGO", "DISTRITO", "DEPARTAMENTO", "MUNICIPIO"]
        target_table = None
        target_index = None

        for i, df in enumerate(tables):
            df_columns_upper = [str(col).upper().strip() for col in df.columns]
            if all(header in df_columns_upper for header in required_headers):
                target_table = df
                target_index = i
                break
            else:
                if len(df) > 0:
                    first_row_upper = [
                        str(cell).upper().strip() for cell in df.iloc[0]
                    ]
                    if all(header in first_row_upper for header in required_headers):
                        df.columns = df.iloc[0]
                        df = df.drop(df.index[0]).reset_index(drop=True)
                        target_table = df
                        target_index = i
                        break

        if target_table is None:
            return {
                "success": False,
                "error": "No table found with required headers.",
            }

        target_table = target_table.dropna(how="all")

        return {
            "success": True,
            "data": target_table,
            "table_index": target_index,
            "total_tables": len(tables),
        }

    except Exception as e:
        return {"success": False, "error": str(e)}


def sanitize_filename(text):
    filename = text.lower().replace(" ", "_")
    return re.sub(r"[^\w_.]", "", filename)


def process_html_files_directory(input_dir, output_dir):
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    files = list(input_path.glob("*.xls"))

    print(f"Found {len(files)} .xls files to process")

    successful_files = []
    failed_files = []
    all_departamentos = defaultdict(list)

    for file_path in files:
        result = parse_html_excel_file(file_path)

        if result["success"]:
            df = result["data"]

            departamentos = []
            for departamento, group in df.groupby("DEPARTAMENTO"):
                filename = f"datos_{sanitize_filename(departamento)}.csv"

                output_file = output_path / filename
                group.to_csv(output_file, index=False)

                departamentos.append(
                    {"name": departamento, "filename": filename, "rows": len(group)}
                )

                all_departamentos[departamento].append(
                    {
                        "source_file": file_path.name,
                        "csv_file": filename,
                        "rows": len(group),
                    }
                )

            successful_files.append(
                {
                    "file": file_path.name,
                    "departamentos": departamentos,
                    "total_rows": len(df),
                }
            )

            print(f"  ✅ Processed {file_path.name}: Created {len(departamentos)} CSVs.")

        else:
            failed_files.append({"file": file_path.name, "error": result["error"]})
            print(f"  ❌ Failed {file_path.name}: {result['error']}")

    print("\n" + "=" * 60)
    print("PROCESSING SUMMARY")
    print("=" * 60)

    print(f"\nTotal files: {len(files)}")
    print(f"✅ Successful: {len(successful_files)}")
    print(f"❌ Failed: {len(failed_files)}")

    # The detailed successful/failed files lists will only be printed if there are successful/failed files.
    # The summary already gives a count, so the repetition for successful files is removed here.
    # The detailed list for failed files remains as it provides useful error messages.
    if failed_files:
        print(f"\n❌ FAILED FILES:")
        for item in failed_files:
            print(f"  - {item['file']}: {item['error']}")

    duplicates = {
        name: sources
        for name, sources in all_departamentos.items()
        if len(sources) > 1
    }
    if duplicates:
        print(f"\n⚠️ DUPLICATE DEPARTAMENTOS:")
        for dept_name, sources in duplicates.items():
            print(f"  {dept_name}: appears in {len(sources)} files")
    else:
        print("\n✅ No duplicate departamentos found")


test_result = parse_html_excel_file("data/raw/establecimiento.xls")
if test_result["success"]:
    print("\nSingle file test successful!")

    print("\n" + "=" * 60)
    print("PROCESSING ALL FILES")
    print("=" * 60)
    process_html_files_directory("data/raw", "data/csv")
else:
    print(f"❌ Single file test failed: {test_result['error']}")


Single file test successful!

PROCESSING ALL FILES
Found 23 .xls files to process
  ✅ Processed establecimiento (10).xls: Created 1 CSVs.
  ✅ Processed establecimiento (11).xls: Created 1 CSVs.
  ✅ Processed establecimiento (12).xls: Created 1 CSVs.
  ✅ Processed establecimiento (13).xls: Created 1 CSVs.
  ✅ Processed establecimiento (14).xls: Created 1 CSVs.
  ✅ Processed establecimiento (15).xls: Created 1 CSVs.
  ✅ Processed establecimiento (16).xls: Created 1 CSVs.
  ✅ Processed establecimiento (17).xls: Created 1 CSVs.
  ✅ Processed establecimiento (18).xls: Created 1 CSVs.
  ✅ Processed establecimiento (19).xls: Created 1 CSVs.
  ✅ Processed establecimiento (1).xls: Created 1 CSVs.
  ✅ Processed establecimiento (20).xls: Created 1 CSVs.
  ✅ Processed establecimiento (21).xls: Created 1 CSVs.
  ✅ Processed establecimiento (22).xls: Created 1 CSVs.
  ✅ Processed establecimiento (2).xls: Created 1 CSVs.
  ✅ Processed establecimiento (3).xls: Created 1 CSVs.
  ✅ Processed establecim

## .csv a DataFrames
Luego de haber creado los archivos, podemos cargarlos a DataFrames para realizar el análisis necesario.

In [50]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter, defaultdict
import matplotlib.pyplot as plt

# Load all CSV files
csv_dir = Path("data/csv")
csv_files = list(csv_dir.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files")

# Load all datasets
datasets = {}
for csv_file in csv_files:
    dataset_name = csv_file.stem
    datasets[dataset_name] = pd.read_csv(csv_file)
    
print(f"Loaded {len(datasets)} datasets")

Found 23 CSV files
Loaded 23 datasets


# Descripción

## Filas y Columnas

In [51]:
# Check shapes of all datasets
shape_info = []
for name, df in datasets.items():
    shape_info.append({
        'dataset': name,
        'rows': df.shape[0],
        'columns': df.shape[1]
    })

shape_df = pd.DataFrame(shape_info)
print("📊 Dataset Shapes:")
print(shape_df.sort_values('rows', ascending=False))

# Summary statistics
print(f"\n📈 Summary:")
print(f"Total rows across all datasets: {shape_df['rows'].sum():,}")
print(f"Average rows per dataset: {shape_df['rows'].mean():.0f}")
print(f"Min/Max rows: {shape_df['rows'].min()} / {shape_df['rows'].max()}")

📊 Dataset Shapes:
                 dataset  rows  columns
19       datos_guatemala  1036       17
16  datos_ciudad_capital   864       17
7       datos_san_marcos   431       17
18       datos_escuintla   393       17
3   datos_quetzaltenango   365       17
14   datos_chimaltenango   300       17
1          datos_jutiapa   296       17
11   datos_suchitepequez   296       17
20   datos_huehuetenango   295       17
22    datos_alta_verapaz   294       17
21          datos_izabal   273       17
5       datos_retalhuleu   272       17
2            datos_peten   270       17
6     datos_sacatepequez   208       17
4           datos_quiche   184       17
15      datos_chiquimula   136       17
8       datos_santa_rosa   133       17
0           datos_jalapa   121       17
9           datos_solola   111       17
17     datos_el_progreso    97       17
10    datos_baja_verapaz    94       17
13          datos_zacapa    70       17
12     datos_totonicapan    51       17

📈 Summary:
Total rows

## Integridad de los Datos

### Consistencia en Nombres de Columnas

In [52]:
# Check if all datasets have the same columns
all_columns = []
column_consistency = {}

for name, df in datasets.items():
    columns = list(df.columns)
    all_columns.append(columns)
    column_consistency[name] = columns

# Check if all column sets are identical
first_columns = all_columns[0]
all_same = all(columns == first_columns for columns in all_columns)

if all_same:
    print(f"\n✅ Standard columns ({len(first_columns)}):")
    for i, col in enumerate(first_columns, 1):
        print(f"  {i:2d}. {col}")
else:
    print("\n❌ Column differences found:")
    for name, columns in column_consistency.items():
        if columns != first_columns:
            print(f"  {name}: {columns}")


✅ Standard columns (17):
   1. CODIGO
   2. DISTRITO
   3. DEPARTAMENTO
   4. MUNICIPIO
   5. ESTABLECIMIENTO
   6. DIRECCION
   7. TELEFONO
   8. SUPERVISOR
   9. DIRECTOR
  10. NIVEL
  11. SECTOR
  12. AREA
  13. STATUS
  14. MODALIDAD
  15. JORNADA
  16. PLAN
  17. DEPARTAMENTAL


### Encoding Problemático
Como mencionamos anteriormente, el encoding de los archivos originales era distinto de "utf-8". Nos dimos cuenta al realizar el análisis sobre el encoding problemático, sin embargo al cambiarlo dentro de la función anterior logramos correr con éxito este análisis.

In [53]:
import pandas as pd
from collections import defaultdict
import re

# Check for encoding issues (corrupted characters)
problematic_char = "�"
all_problematic_samples = defaultdict(list)
issue_found = False

for dataset_name, df in datasets.items():
    for col in df.columns:
        if df[col].dtype == "object":  # Only check text columns
            str_series = df[col].astype(str) 
            
            # Identify rows with the problematic character '�'
            contains_char_mask = str_series.str.contains(problematic_char, na=False)
            
            if contains_char_mask.any(): # If any '�' found in this column
                issue_found = True
                # Get unique problematic values, up to 5, for this column
                current_samples = str_series[contains_char_mask].unique().tolist()
                for sample_val in current_samples:
                    if len(all_problematic_samples[col]) < 5:
                        all_problematic_samples[col].append(sample_val)


if issue_found:
    print(f"❌ Encoding issues found (char: '{problematic_char}'):")
    # Sort columns by name for consistent output
    sorted_cols_with_issues = sorted(all_problematic_samples.keys()) 

    for col in sorted_cols_with_issues:
        samples = all_problematic_samples[col]
        print(f"  Column '{col}':")
        for val in samples:
            print(f"    • {val}")
else:
    print(f"✅ No encoding issues found (char: '{problematic_char}')")

✅ No encoding issues found (char: '�')


# Análisis de Variables
Las variables que más operaciones de limpieza necesitan son:

## Código
Este valor parece ser un identificador único, queremos explorar las siguientes propiedades:
- Unicidad: Este código es único dentro de su respectivo dataset o todos?
- Formato: Es consistente el formato en todos los datasets? Existen errores de digitación?
- Valores Faltantes: Existen valores faltantes?
- Nos ayuda a identificar únicamente algún otro valor?

Debido a esto, queremos realizar los siguientes pasos de limpieza:

- Identificar unicidad del código
- Identificar Valores Faltantes
- Revisar errores de digitación, cómo lo pueden ser whitespaces o formato incongruente
- Identificar si nos puede ayudar a identificar otras variables para verificar errores de digitación

## Distrito
Este valor parece ser un identificador geográfico, siguiendo un formato XX-YYY. Queremos explorar las siguientes propiedades
- Formato: Es consistente el formato?
- Valores Faltantes: Existen valores faltantes dentro de los datasets?
- Nos ayuda a identificar únicamente algún otro valor?

Debido a esto, queremos realizar los siguientes pasos de limpieza:

- Identificar valores faltantes
- Revisar errores de digitación
- Enforzar un formato consistente
- Identificar si nos puede ayuda a verificar la consistencia de Municipio o algún otro valor

## Departamento

Ya que los DataFrames se encuentran divididos por departamento, esta entrada debería ser completamente consistente. Además, podemos proponer los siguientes pasos para una mayor consistencia:

- Identificar el valor RAW más común, ya que un error de digitación sería menos frecuente
- Tomar ese valor y realizar las siguientes transformaciones
    - Conversión a minúsculas
    - Reemplazo de espacios por _
    - Aplicar a todas las columnas de cada DF individual
- Aplicar OHE luego de mergear los DFs